# GDC

In [3]:
#%%
import pandas as pd
import glob
import os

sample_sheet = pd.read_csv('D:/data/RNASeq/GDC/sample_sheet.tsv', sep='\t')
csv_name = "gdcdataset"
csv_path = "D:/data/RNASeq/dataset/"

paths = glob.glob('D:\\data\\RNASeq\\GDC\\gdc_download_20220707_111311.305944\\*')

for SPLIT in range(10):
    labels = []
    runs = []
    paths_array = []
    sample_ids = []
    case_ids = []

    case_id = open(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\splits\\testsplit{SPLIT}.txt", "r").read().split()
    split_file_ids = list(sample_sheet[sample_sheet["Case ID"].isin(case_id)]["File ID"])
    for path in paths:
        file_id = os.path.split(path)[-1]
        if file_id in split_file_ids:
            if not os.path.exists(path) or not os.path.isdir(path) or len(os.listdir(path)) == 0:
                print("Path does not exist or empty ", path)
                pass
            else:
                for file in glob.glob(path + r"\*"+".tsv"):
                    file_name = os.path.split(file)[-1]
                    paths_array.append(path)
                    runs.append(file_name)
                    labels.append(sample_sheet[sample_sheet["File ID"] == file_id]["Sample Type"].iloc[0])
                    sample_ids.append(sample_sheet[sample_sheet["File ID"] == file_id]["Sample ID"].iloc[0])
                    case_ids.append(sample_sheet[sample_sheet["File ID"] == file_id]["Case ID"].iloc[0])

    # create dataset for knowseq
    mergedCounts = pd.DataFrame()
    mergedCounts['Path'] = paths_array
    mergedCounts['Run'] = runs
    mergedCounts['Class'] = labels
    mergedCounts['Sample ID'] = sample_ids
    mergedCounts['Case ID'] = case_ids

    mergedCounts.to_csv(csv_path+"labels/"+csv_name+ str(SPLIT) +'.csv', sep=',', index=False)

In [4]:
# Knowseq does not work with STAR count files from GDC, therefore a full matrix must be created 
# COLUMS=> SAMPLE_ID ROW=> GEN_ID The labels can be taken using R from the given .csv file
# lo que devuelve Knoqseq son dos columnas $countsMatrix (X) y $labels(y)

n_splits = 10

dataGtex = pd.read_csv("D:/data/RNASeq/GTEx/gene_reads_pancreas.gct", sep="\t", header=2)

dataGtex.index = dataGtex["Name"]
dataGtex.drop(columns=["id","Description", "Name"], inplace=True)


for SPLIT in range(n_splits):
    print("Split: ", SPLIT)
    countsMatrix = []
    
    TRAIN_SPLITS = list(range(n_splits))
    # We take out the SPLIT and SPLIT+1 sets for val and testing
    if SPLIT == n_splits-1: # For the final split for validation we take the firs one for test
        TRAIN_SPLITS.remove(0) 
    else:
        TRAIN_SPLITS.remove(SPLIT+1)
    TRAIN_SPLITS.remove(SPLIT)

    for TRAIN_SPLIT in TRAIN_SPLITS:
        mergedCounts = pd.read_csv(csv_path+"labels/"+csv_name+str(TRAIN_SPLIT)+'.csv', sep=',')

        for sample_id, path, name, label in zip(mergedCounts['Sample ID'], mergedCounts['Path'], mergedCounts['Run'], mergedCounts['Class']):
            counts = pd.read_csv(path+"\\"+name, sep='\t', header=1)#["unstranded"]
            counts = counts.set_index("gene_id")["unstranded"]
            counts.name = sample_id
            countsMatrix.append(counts)

    countsMatrix = pd.concat(countsMatrix, axis=1) # All GDC splits are concatenated

    #countsMatrix.to_csv(csv_path+'countsMatrix/countsMatrixGDC_split'+str(SPLIT)+'.csv', sep=',')

    # Selecting all samples that have a Case ID in belonging to the corresponding split
    case_id = open(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\splits\\trainsplit{SPLIT}.txt", "r").read().split()
    split_sample_ids = [sample_id for sample_id in dataGtex.columns if ("-".join(sample_id.split("-")[:2]) in case_id)]
    dataGtex_split = dataGtex[split_sample_ids]

    gtexDataset = pd.DataFrame({"Case ID": ["-".join(sample_id.split("-")[:2]) for sample_id in list(dataGtex_split.columns)],
                                "Sample ID":list(dataGtex_split.columns), 
                                "Class":["Solid Tissue Normal" for i in dataGtex_split.columns]})

    #data.to_csv(csv_path + 'countsMatrixGTEx.csv', sep=',')

    gtexDataset.to_csv(csv_path + 'labels/gtexdataset'+str(SPLIT)+'.csv', sep =',')

    countsMatrix = pd.concat([countsMatrix, dataGtex_split], axis=1) # Concatenating all GTEx and GDC data for the split

    #Deleting double (sexual) genes that are not expressed
    for row in countsMatrix.iterrows():
        not_expressed = all(v == 0 for v in list(row[1]))
        if "PAR_Y" in row[0] and not_expressed:
            countsMatrix.drop(index=row[0], inplace=True)

    countsMatrix.index = [index.split(".")[0] for index in countsMatrix.index]

    countsMatrix.fillna(0, inplace=True)

    countsMatrix.to_csv(csv_path + 'countsMatrix/countsMatrixGDC+GTEX_split'+str(SPLIT)+'.csv', sep=',')

Split:  0
Split:  1
Split:  2
Split:  3
Split:  4
Split:  5
Split:  6
Split:  7
Split:  8
Split:  9
